# Cryptostock Analytica

In [ ]:
# Initial imports
import os
import time
import requests
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import hvplot.pandas
import holoviews as hv
import numpy as np

%matplotlib inline

### Function definitions

In [ ]:
# Fetch stock data for a given stock symbol and date range
def fetch_stock_data(stock_symbol, start_date, end_date):
    stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)
    stock_data['Date'] = pd.to_datetime(stock_data['Date'], utc=True)
    return stock_data

# Fetch the top N cryptocurrencies by market cap
def fetch_top_n_cryptos(n=10):
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': n,
        'page': 1,
        'sparkline': False,
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data

# Fetch historical price data for a specific cryptocurrency using its coin_id
def fetch_crypto_data(coin_id, from_timestamp, to_timestamp):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"
    params = {
        'vs_currency': 'usd',
        'from': from_timestamp,
        'to': to_timestamp
    }
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data['prices'], columns=['time', 'price'])
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    return df

# Get the historical data of a specific cryptocurrency by its name
def get_data_by_name(name):
    for crypto in top_cryptos:
        if crypto['name'].lower() == name.lower():
            coin_id = crypto['id']
            return crypto_data[coin_id]
    return None

# Calculate the Pearson correlation between two datasets
def pearson_correlation(crypto, stocks):
    return pd.Series(crypto).corr(pd.Series(stocks))

# Interpret the correlation value
def interpret_correlation(correlation):
    if correlation >= 0.7:
        interpretation = "strong positive correlation"
    elif correlation >= 0.3:
        interpretation = "moderate positive correlation"
    elif correlation >= -0.3:
        interpretation = "weak correlation"
    elif correlation >= -0.7:
        interpretation = "moderate negative correlation"
    else:
        interpretation = "strong negative correlation"
    return interpretation

### Data Acquisition

In [ ]:
# Stock data acquisition
# Ask user for the stock symbol
selected_stock = input("Enter the stock symbol you want to view data for: ").upper()

end_date = pd.Timestamp.now(tz='UTC')
start_date = end_date - pd.DateOffset(years=3)

# Fetch the stock data
selected_stock_data = fetch_stock_data(selected_stock, start_date, end_date)

# Print the stock data
if not selected_stock_data.empty:
    # Round the price data to 2 decimal places
    selected_stock_data[['Open', 'High', 'Low', 'Close']] = selected_stock_data[['Open', 'High', 'Low', 'Close']].round(2)

    # Print the data
    print(selected_stock_data)
else:
    print("Error: Invalid stock symbol or no data available.")

selected_stock_data.set_index('Date')

In [ ]:
# Cryptocurrency data acquisition
# Unix timestamps for three years of data
to_timestamp = pd.Timestamp.now(tz='UTC').timestamp()
from_timestamp = pd.Timestamp.now(tz='UTC') - pd.DateOffset(years=3)
from_timestamp = from_timestamp.timestamp()

# Fetch the top 10 cryptocurrencies by market cap
top_cryptos = fetch_top_n_cryptos(10)

# Fetch historical data for each of the top 10 cryptos
crypto_data = {}
for crypto in top_cryptos:
    coin_id = crypto['id']
    try:
        df = fetch_crypto_data(coin_id, from_timestamp, to_timestamp)
        crypto_data[coin_id] = df
        print(f"Fetched data for {coin_id}")
        time.sleep(2)  # Add a delay between requests to avoid rate limiting
    except Exception as e:
        print(f"Error fetching data for {coin_id}: {e}")
        
# Ask user for the name of the cryptocurrency
selected_crypto = input("Enter the name of the cryptocurrency you want to view data for: ")

# Get the data for the selected cryptocurrency
selected_crypto_data = get_data_by_name(selected_crypto)
    
# Print the cryptocurrency data
if selected_crypto_data is not None:
    # Round the price data to 2 decimal places
    selected_crypto_data['price'] = selected_crypto_data['price'].round(2)

    # Print the data
    print(selected_crypto_data)
else:
    print("Error: Invalid cryptocurrency name.")

selected_crypto_data.set_index('time')

 ## Part 1 - Correlation Analysis

In [ ]:
# Calculate the correlation between crypto and stock prices
correlation = pearson_correlation(selected_crypto_data['price'], selected_stock_data['Close'])
print(correlation)

# Create a DataFrame containing crypto and stock prices
combined_data = pd.DataFrame({
    'Crypto Prices': selected_crypto_data['price'],
    'Stock Prices': selected_stock_data['Close']
})

# Create a scatter plot using Hvplot
scatter_plot = combined_data.hvplot.scatter(
    x='Crypto Prices',
    y='Stock Prices',
    title='Scatter Plot of Crypto Prices vs. Stock Prices'
)

# Position the correlation label on the scatter plot
x_position = 0.85 * combined_data['Crypto Prices'].max()
y_position = 0.15 * combined_data['Stock Prices'].max()
correlation_label = hv.Text(x_position, y_position, f'Correlation: {correlation:.2f}', fontsize=14)

# Get the textual interpretation of the correlation
correlation_interpretation = interpret_correlation(correlation)
print(f"The assets have a {correlation_interpretation}.")

# Position the interpretation label on the scatter plot
interpretation_label = hv.Text(x_position, y_position - y_position * 0.1, f'Interpretation: {correlation_interpretation}', fontsize=12)

# Combine the scatter plot, correlation label, and interpretation label
plot_with_label_and_interpretation = scatter_plot * correlation_label * interpretation_label

# Display the plot
plot_with_label_and_interpretation.opts(legend_position='top_left', height=400, width=900)

## Part 2 - Technical Analysis

### Moving Averages

In [ ]:
## YOUR CODE HERE

### Relative Strength Index (RSI)

In [ ]:
## YOUR CODE HERE

### Bollinger Bands

In [ ]:
## YOUR CODE HERE

### Standard Deviation

In [ ]:
# Create a DataFrame of daily returns for the selected cryptocurrency and stock
stock_and_crypto_returns_df = pd.DataFrame({
    'Crypto Prices': selected_crypto_data['price'].pct_change().dropna(),
    'Stock Prices': selected_stock_data['Close'].pct_change().dropna()
})

# Calculate the annualized standard deviation for the cryptocurrency and stock
crypto_std_annualized = np.sqrt(252) * stock_and_crypto_returns_df['Crypto Prices'].std()
stocks_std_annualized = np.sqrt(252) * stock_and_crypto_returns_df['Stock Prices'].std()

# Create a DataFrame of the annualized standard deviation for the cryptocurrency and stock
assets_annualized_std = pd.DataFrame({
    "Asset type" : ['Crypto' , 'Stock'],
    "Annualized Standard Deviation": [crypto_std_annualized , stocks_std_annualized]
})

# Create a bar plot of the annualized standard deviation for the cryptocurrency and stock using Hvplot
assets_annualized_std.hvplot.bar(
    title= "Annualized Standard Deviation",
    x="Asset type",
    y="Annualized Standard Deviation"
)

## Part 3 - Risk Analysis

### Volatility

In [ ]:
## YOUR CODE HERE

### Value at Risk (VaR)

In [ ]:
## YOUR CODE HERE